In [69]:
#importing libraries

import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
import random
from sklearn.model_selection import train_test_split
from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Embedding, LSTM, RepeatVector, TimeDistributed

In [3]:
#to disable TF 2.x behaviour
tf.compat.v1.disable_v2_behavior()

#enables eager execution
tf.compat.v1.enable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
df = pd.read_csv("C:/Users/user8/Downloads/MGP Resources/hindi_english_parallel.csv")
df

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
...,...,...
1561836,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...
1561838,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...
1561839,खाद्य और जल सुरक्षा; पर्यावरण की दृष्टि से वहन...,्त है। Issues such as food and water security;...


In [4]:
#no of records where the hindi and english columns have same string

c=0; same_rec_l=[]
for i in range(0,1561841):
    if(df['hindi'][i]==df['english'][i]):
        c+=1
        same_rec_l.append(i)
    
print(c)

3468


In [5]:
for i in same_rec_l:
    print(i, '\t', df['hindi'][i], '\t', df['english'][i])

42 	 0, 0 	 0, 0
235 	 0, 0 	 0, 0
438 	 0, 0 	 0, 0
636 	 0, 0 	 0, 0
846 	 0, 0 	 0, 0
1047 	 0, 0 	 0, 0
1247 	 0, 0 	 0, 0
1449 	 0, 0 	 0, 0
1834 	 9 	 9
1917 	 % d:% 02d 	 % d:% 02d
2488 	 9 	 9
2571 	 % d:% 02d 	 % d:% 02d
3129 	 % d:% 02d 	 % d:% 02d
3211 	 9 	 9
3659 	 9 	 9
3742 	 % d:% 02d 	 % d:% 02d
4225 	 9 	 9
4305 	 % d:% 02d 	 % d:% 02d
4803 	 9 	 9
4886 	 % d:% 02d 	 % d:% 02d
5433 	 9 	 9
5516 	 % d:% 02d 	 % d:% 02d
7397 	 Scratchbox 	 Scratchbox
7410 	 _ Valgrind 	 _ Valgrind
7418 	 Cachegrind 	 Cachegrind
7424 	 Helgrind 	 Helgrind
7753 	 GTKmm 	 GTKmm
7758 	 Generic gcj compiled java 	 Generic gcj compiled java
7774 	 Xlib 	 Xlib
7775 	 Xlib dock 	 Xlib dock
7789 	 SamplePlugin 	 SamplePlugin
7843 	 Valgrind 	 Valgrind
7844 	 Memcheck Addrcheck Helgrind 	 Memcheck Addrcheck Helgrind
7846 	 Copyright (c) Naba Kumar 	 Copyright (c) Naba Kumar
7851 	 Default Text below icons Text beside icons Icons only Text only 	 Default Text below icons Text beside icons Icons on

In [6]:
df = df.drop(same_rec_l).reset_index(drop=True)

In [7]:
c=0;
for i in range(0,1561841-len(same_rec_l)):
    if(df['hindi'][i]==df['english'][i]):
        c+=1
    
print(c)

#so we confirm that we have no records with both cols same

0


In [8]:
def clean_text(sample, Hindi=True):
    
    sample = " ".join([x.lower() for x in sample.split()])
    sample = re.sub('\(', '', sample)
    sample = re.sub('।', '', sample)
    sample = re.sub('\)', '', sample)
    sample = re.sub('[%s]' % re.escape(string.punctuation), ' ', sample) #punctuations
    sample = re.sub(r'\n', ' ', sample) #new line character
    sample = re.sub(r'\\n', ' ', sample) #new line character
    sample = re.sub("[''""...“”‘’…]", '', sample) #list of quotation marks
    sample = " ".join(x.strip() for x in sample.split()) #strips whitespace
    
    if(Hindi):
        sample = ' '.join(list(filter(lambda ele: re.search("[ऀ-ॿ\s]+", ele) is not None, sample.split()))) #languages other than english
    else:
        sample = ' '.join(list(filter(lambda ele: re.search("[a-zA-Z\s]+", ele) is not None, sample.split()))) #languages other than english
    
    sample = sample.strip()
    sample = ' '.join([x.strip()  for x in sample.split()])
    
    return sample

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558373 entries, 0 to 1558372
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   hindi    1552317 non-null  object
 1   english  1557648 non-null  object
dtypes: object(2)
memory usage: 23.8+ MB


In [10]:
df = df.dropna().reset_index(drop=True)
df

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default
...,...,...
1552254,Members making oath/affirmation,शपथ लेने/प्रतिज्ञान करने वाले सदस्य
1552255,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1552256,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...
1552257,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...


In [11]:
df['cleaned_hindi'] = df['hindi'].apply(lambda x: clean_text(str(x), Hindi=True))
df

,hindi,english,cleaned_hindi
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग इन खाका
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default,उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से नि...
...,...,...,...
1552254,Members making oath/affirmation,शपथ लेने/प्रतिज्ञान करने वाले सदस्य,
1552255,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण –जहां इस उपधारा के अधीन हानि और लाभ...
1552256,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...
1552257,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...


In [12]:
for i in range(0,10):
    j = random.randint(0,1552259)
    print(j, "\t", df['cleaned_hindi'][j], "\n")

343725 	 तुम इस भय से छिपते न थे कि तुम्हारे कान तुम्हारे विरुद्ध गवाही देंगे और न इसलिए कि तुम्हारी आँखें गवाही देंगी और न इस कारण से कि तुम्हारी खाले गवाही देंगी बल्कि तुमने तो यह समझ रखा था कि अल्लाह तुम्हारे बहुत से कामों को जानता ही नहीं 

251336 	 जहन्नुम की वह भड़कती आग है कि खाल उधेड़ कर रख देगी 

120091 	 प्रशांत पलाऊ 

1121686 	 राज्य से देश के निर्यात का प्रतिशत कपड़ा निर्यात होता है 

410145 	 और जब नाप तौल कर देना हो तो पैमाने को पूरा भर दिया करो और जब तौल कर देना हो तो बिल्कुल ठीक तराजू से तौला करो मामले में यही तरीक़ा बेहतर है और अन्जाम भी उसका अच्छा है 

1299372 	 स्तम्भ भी गोलाकार तश्तरियों और हरे से परतदार शिलाविशेष के बने हुए हैं जिन्हें एक के ऊपर एक को रखकर वैसे ही मसाले से जोङा गया है 

360171 	 और सारे आसमान व ज़मीन की ग़ैब की बातें ख़ुदा ही के लिए मख़सूस हैं और ख़ुदा क़यामत का वाकेए होना तो ऐसा है जैसे पलक का झपकना बल्कि इससे भी जल्दी बेशक ख़ुदा हर चीज़ पर क़ुदरत कामेला रखता है 

284391 	 तब जिन लोगों को अपनी दौलत दुनिया पर घमन्ड था कहने लगे हम तो जिस पर तुम ईमान

In [13]:
c=0; eng_in_hin_l=[]
for i in range(0,1552259):
    if(df['cleaned_hindi'][i]==""):
        c+=1
        eng_in_hin_l.append(i)
    
print(c,"\n")
print(eng_in_hin_l)

17462 

[527, 936, 1133, 1332, 1533, 1711, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1905, 1906, 2363, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2557, 2558, 3114, 3115, 3130, 3184, 3185, 3188, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198, 3199, 3200, 3201, 3530, 3637, 3638, 3639, 3640, 3641, 3642, 3643, 3644, 3645, 3646, 3647, 3648, 3724, 3725, 4095, 4201, 4202, 4203, 4204, 4205, 4206, 4207, 4208, 4209, 4210, 4211, 4212, 4285, 4286, 4670, 4777, 4778, 4779, 4780, 4781, 4782, 4783, 4784, 4785, 4786, 4787, 4788, 4864, 4865, 5298, 5405, 5406, 5407, 5408, 5409, 5410, 5411, 5412, 5413, 5414, 5415, 5416, 5492, 5493, 5986, 5988, 6099, 6736, 6737, 6738, 6751, 6752, 6755, 7316, 7352, 7373, 7492, 7493, 7494, 7495, 7721, 7729, 7730, 7731, 7847, 7848, 7849, 7875, 7913, 7915, 7916, 7929, 8048, 8049, 8050, 8051, 8276, 8284, 8285, 8286, 8402, 8403, 8404, 8429, 8473, 8523, 8524, 8525, 8526, 8669, 8680, 8681, 8729, 8740, 8767, 8768, 883

In [16]:
c=0
for i in range(0,1552259):
    if(c>25):
        break
    if i in eng_in_hin_l:
        print(i, "\t", df['hindi'][i], "\t", df['english'][i])
        c+=1

527 	 <dead>  	 < dead >
936 	 <dead>  	 < dead >
1133 	 <dead>  	 < dead >
1332 	 <dead>  	 < dead >
1533 	 <dead>  	 < dead >
1711 	 % s is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.  	 % s is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
1818 	 Acard symbol 	 A
1819 	 2number 	 2
1820 	 3number 	 3
1821 	 4number 	 4
1822 	 5number 	 5
1823 	 6number 	 6
1824 	 7number 	 7
1825 	 8number 	 8
1826 	 Jcard symbol 	 J
1827 	 Qcard symbol 	 Q
1828 	 Kcard symbol 	 K
1829 	 1number 	 1
1905 	 % d% d "in this string this to"% Id% d% d%%%%% "(U + 0025 PERCENT SIGN) you do NOT need to escape it with another"%  	 % d
1906 	 % d%% Not ApplicableNot Appli

In [17]:
df = df.drop(eng_in_hin_l, axis=0).reset_index(drop=True)
df

,hindi,english,cleaned_hindi
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग इन खाका
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default,उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से नि...
...,...,...,...
1534792,भारत अपनी 64 प्रतिशत कार्यशील आयु समूह आबादी क...,"प्राप्त होगी। By 2020, India is set to become ...",भारत अपनी प्रतिशत कार्यशील आयु समूह आबादी के स...
1534793,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण –जहां इस उपधारा के अधीन हानि और लाभ...
1534794,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...
1534795,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...


In [18]:
df['cleaned_english'] = df['english'].apply(lambda x: clean_text(str(x), Hindi=False))
df

,hindi,english,cleaned_hindi,cleaned_english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग इन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका,the default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default,उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से नि...,a list of plugins that are disabled by default
...,...,...,...,...
1534792,भारत अपनी 64 प्रतिशत कार्यशील आयु समूह आबादी क...,"प्राप्त होगी। By 2020, India is set to become ...",भारत अपनी प्रतिशत कार्यशील आयु समूह आबादी के स...,by india is set to become the worlds youngest ...
1534793,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...,स्पष्टीकरण –जहां इस उपधारा के अधीन हानि और लाभ...,
1534794,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,है। I note that this is a landmark meeting – n...,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,i note that this is a landmark meeting not onl...
1534795,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,है। In the presentations that they made before...,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,in the presentations that they made before me ...


In [19]:
df = df.drop(['hindi', 'english'], axis=1)
df

,cleaned_hindi,cleaned_english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग इन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका,the default plugin layout for the top panel
4,उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से नि...,a list of plugins that are disabled by default
...,...,...
1534792,भारत अपनी प्रतिशत कार्यशील आयु समूह आबादी के स...,by india is set to become the worlds youngest ...
1534793,स्पष्टीकरण –जहां इस उपधारा के अधीन हानि और लाभ...,
1534794,मैंने गौर किया है कि यह न केवल अपने महत्त्वपूर...,i note that this is a landmark meeting not onl...
1534795,उन्होंने मेरे समक्ष जो प्रदर्शन किया उसमें से ...,in the presentations that they made before me ...


In [50]:
df.to_csv("C:/Users/user8/Downloads/MGP Resources/Project 12/data/hin_eng_cleaned_dataset.csv")

KeyboardInterrupt: 

In [147]:
if True:
    df = pd.read_csv("C:/Users/user8/Downloads/MGP Resources/Project 12/data/hin_eng_cleaned_dataset.csv")
    df = df.drop(df.columns[0], axis=1)

    
df = df.drop([i for i in range(50001, len(df))])
y = df['cleaned_hindi'] #hindi column as X
X = df['cleaned_english'] #english column as y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)
X_train.shape, X_test.shape

((37500,), (12501,))

In [148]:
df['cleaned_hindi'] = df['cleaned_hindi'].apply(lambda x: str(x))
df['cleaned_english'] = df['cleaned_english'].apply(lambda x: str(x))

In [149]:
# df['length_eng_sentence']=df['cleaned_english'].apply(lambda x:len(x.split(" ")))
# df['length_hin_sentence']=df['cleaned_hindi'].apply(lambda x:len(x.split(" ")))

In [150]:
max_length_src = 100 #max(df['length_hin_sentence'])
max_length_tar = 100 #max(df['length_eng_sentence']) 

max_length_src, max_length_tar

(100, 100)

In [151]:
def generate_batch(X = X_train, y = y_train, batch_size = 64):
    
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [152]:
def tokenize(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

In [153]:
vocab_eng = Counter()
for text in df['cleaned_english']:
    tokens = [word for word in text.split()]
    vocab_eng.update(tokens)
    
print(vocab_eng.most_common(50))

[('the', 12787), ('to', 6533), ('of', 4785), ('a', 4233), ('s', 3648), ('file', 2650), ('project', 2161), ('in', 2142), ('for', 2137), ('not', 1996), ('be', 1829), ('is', 1753), ('you', 1613), ('disc', 1581), ('files', 1376), ('and', 1338), ('or', 1168), ('this', 1159), ('on', 1046), ('name', 1030), ('with', 990), ('add', 975), ('please', 970), ('image', 944), ('new', 924), ('an', 918), ('use', 913), ('plugin', 855), ('it', 826), ('no', 824), ('from', 812), ('will', 805), ('remove', 798), ('all', 795), ('select', 783), ('selected', 781), ('video', 778), ('do', 691), ('if', 691), ('whether', 682), ('current', 676), ('directory', 668), ('error', 665), ('are', 658), ('move', 657), ('show', 646), ('cd', 643), ('open', 637), ('could', 636), ('want', 632)]


In [154]:
vocab_set_eng = set([x for x in vocab_eng if len(x)>2])

tokenizer_eng = tokenize(X_train.astype('str').values)
vocab_size_eng = len(tokenizer_eng.word_index) + 1
max_length_eng = max([len(s.split()) for s in X_train.astype('str')])

In [155]:
num_encoder_tokens = len(vocab_set_eng)

In [156]:
vocab_hin = Counter()
for text in df['cleaned_hindi']:
    tokens = [word for word in text.split()]
    vocab_hin.update(tokens)
    
print(vocab_hin.most_common(50))

[('के', 6549), ('को', 5686), ('है', 5668), ('करें', 4530), ('नहीं', 3621), ('का', 3610), ('में', 3473), ('लिए', 3035), ('से', 2948), ('फ़ाइल', 2655), ('की', 2367), ('पर', 1856), ('डिस्क', 1759), ('रहा', 1447), ('एक', 1407), ('और', 1258), ('या', 1140), ('परियोजना', 1098), ('क्या', 1046), ('हैं', 1032), ('छवि', 1022), ('नाम', 1016), ('यह', 996), ('कर', 934), ('फाइल', 932), ('आप', 926), ('कोई', 878), ('कृपया', 865), ('जोड़ें', 849), ('किया', 834), ('साथ', 833), ('करने', 826), ('प्रयोग', 786), ('इंच', 765), ('जा', 764), ('सभी', 721), ('नया', 705), ('चुनें', 674), ('वीडियो', 673), ('चयनित', 639), ('गया', 632), ('स्थान', 598), ('प्लगइन', 596), ('इस', 595), ('खाली', 593), ('त्रुटि', 590), ('निर्देशिका', 589), ('जाएँ', 573), ('स्थिति', 555), ('योग्य', 541)]


In [157]:
vocab_set_hin = set([x for x in vocab_hin if len(x)>2])

tokenizer_hin = tokenize(y_train.astype('str').values)
vocab_size_hin = len(tokenizer_hin.word_index) + 1
max_length_hin = max([len(s.split()) for s in y_train.astype('str')])

In [158]:
num_decoder_tokens = len(vocab_set_hin)

In [159]:
input_words = sorted(list(vocab_set_eng))
target_words = sorted(list(vocab_set_hin))

input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [160]:
latent_dim=100

In [161]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [162]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [163]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_21 (Embedding)       (None, None, 100)    349100      ['input_19[0][0]']               
                                                                                                  
 embedding_22 (Embedding)       (None, None, 100)    397500      ['input_20[0][0]']               
                                                                                            

In [164]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = 64),
                    steps_per_epoch = len(X_train)//64,
                    epochs=10,
                    validation_data = generate_batch(X_test, y_test, batch_size = 64),
                    validation_steps = len(X_test)//64)

# model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

C:\Users\user8\AppData\Local\Temp\ipykernel_3132\1747619817.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = 64),
C:\Users\user8\AppData\Local\Temp\ipykernel_3132\3116588519.py:8: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):


KeyError: 'in'